In [111]:
from nltk.corpus import conll2002
from sklearn.feature_extraction import DictVectorizer
from nltk.stem.snowball import SpanishStemmer
import re
import pycrfsuite

# Helper Function for Word Shape

In [112]:
def word_shape(word):
    word = re.sub('[A-Z]|[À-Ú]', 'X', word)
    word = re.sub('[a-z]|[à-ú]', 'x', word)
    word = re.sub('[0-9]', 'd', word)
    return word

In [113]:
def short_word_shape(word):
    shape = word_shape(word)
    
    short_shape = ''
    prev_shape = ''
    is_punctuation = False

    for i in range(len(shape)):
        if shape[i] == 'X':
            curr_shape = 'X'
        elif shape[i] == 'x':
            curr_shape = 'x'
        elif shape[i] == 'd':
            curr_shape = 'd'
        else:
            is_punctuation = True
            curr_shape = shape[i]
        if is_punctuation or curr_shape != prev_shape:
            short_shape += curr_shape
            
        is_punctuation = False
        prev_shape = curr_shape
        
    return short_shape

# Get Features

In [114]:
def getfeats(word, pos, o):
    """ This takes the word in question and
    the offset with respect to the instance
    word """
    
    has_hyphen = 0
    if "-" in word:
        has_hyphen = 1
        
    has_apostrophe = 0
    if "'" in word:
        has_apostrophe = 1
        
    stemmer = SpanishStemmer() 
    
    o = str(o)
    features = [
        (o + 'word', word),
        (o + 'pos', pos),
        (o + 'prefix1', word[:1]), 
        (o + 'prefix2', word[:2]),
        (o + 'prefix3', word[:3]),
        (o + 'prefix4', word[:4]),
        (o + 'suffix1', word[-1:]), 
        (o + 'suffix2', word[-2:]), 
        (o + 'suffix3', word[-3:]), 
        (o + 'suffix4', word[-4:]), 
        (o + 'is_upper', word.isupper()),
        (o + 'is_title', word.istitle()),
        (o + 'is_digit', word.isdigit()),
        (o + 'has_hypen', has_hyphen),
        (o + 'has_apostrophe', has_apostrophe),
        (o + 'spanich_stem', stemmer.stem(word)),
        (o + 'word_shape', word_shape(word)),
        (o + 'short_word_shape', short_word_shape(word))
    ]
    #print(features)
    return features

In [79]:
def word2features(sent, i):
    """ The function generates all features
    for the word at position i in the
    sentence."""
    features = []
    # the window around the token
    #for o in [-1,0,1]:
    for o in [-2,-1,0,1,2]:
        if i+o >= 0 and i+o < len(sent):
            word = sent[i+o][0]
            pos = sent[i+o][1]
            featlist = getfeats(word, pos, o)
            if o==0:
                if i == 0:
                    featlist.append(("beginning", True))
                elif i == len(sent)-1:
                    featlist.append(("ending", True))
            
            features.extend(featlist)
    
    return dict(features)

# Get Data

In [115]:
train_sents = list(conll2002.iob_sents('esp.train'))
dev_sents = list(conll2002.iob_sents('esp.testa'))
test_sents = list(conll2002.iob_sents('esp.testb'))

# Add Features to Training Data

In [116]:
train_feats = []
train_labels = []

In [117]:
for sent in train_sents:
    local_feats = []
    loca_labels = []
    for i in range(len(sent)):
        feats = word2features(sent,i)
        local_feats.append(feats)
        loca_labels.append(sent[i][-1])
    train_feats += [local_feats]
    train_labels += [loca_labels]

# Try Different Parameters in CRF

In [128]:
trainer = pycrfsuite.Trainer(verbose=False)

In [129]:
for xseq, yseq in zip(train_feats, train_labels):
    trainer.append(xseq, yseq)

In [ ]:
#c1 = 0
c1 = 0.1

In [ ]:
#c2 = 0
c2 = 0.1

In [ ]:
max_iterations = 100
#max_iterations = 1000

In [130]:
trainer.set_params({
    'c1': c1,   
    'c2': c2,  
    'max_iterations': max_iterations,  
    'feature.possible_transitions': True
})

In [ ]:
train_algo = 'lbfgs'
#train_algo = 'l2sgd'
#train_algo = 'ap'
#train_algo = 'pa'
#train_algo = 'arow'

In [131]:
trainer.select(train_algo)

In [132]:
trainer.train('conll2002-esp.crfsuite')

In [133]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

In [136]:
test_feats = []
test_labels = []

In [137]:
for sent in dev_sents:
    local_feats = []
    loca_labels = []
    for i in range(len(sent)):
        feats = word2features(sent,i)
        local_feats.append(feats)
        loca_labels.append(sent[i][-1])
        
    test_feats += [local_feats]
    test_labels += [loca_labels]  

In [138]:
y_pred = [tagger.tag(xseq) for xseq in test_feats]

In [139]:
with open("results.txt", "w") as out:
    for i in range(len(dev_sents)): 
        sent = dev_sents[i]
        for j in range(len(sent)):
            word = sent[j][0]
            gold = sent[j][-1]
            pred = y_pred[i][j]
            out.write("{}\t{}\t{}\n".format(word,gold,pred))
    out.write("\n")

# Try to Find best C1 and C2

In [99]:
for i in [0.1, 0.2, 0.3, 0.4, 0.5]:
    for j in [0.1, 0.2, 0.3, 0.4, 0.5]:
        trainer = pycrfsuite.Trainer(verbose=False)
        
        for xseq, yseq in zip(train_feats, train_labels):
            trainer.append(xseq, yseq)
            
        trainer.set_params({
            'c1': i,   
            'c2': j,  
            'max_iterations': 100,  
            'feature.possible_transitions': True
            })
        trainer.select('lbfgs')
        trainer.train('conll2002-esp.crfsuite')
        tagger = pycrfsuite.Tagger()
        tagger.open('conll2002-esp.crfsuite')
        y_pred = [tagger.tag(xseq) for xseq in test_feats]
        
        filename = "results_" + str(i) + "_" + str(j) + ".txt"
        
        with open(filename, "w") as out:
            for i in range(len(dev_sents)): 
                sent = dev_sents[i]
                for j in range(len(sent)):
                    word = sent[j][0]
                    gold = sent[j][-1]
                    pred = y_pred[i][j]
                    out.write("{}\t{}\t{}\n".format(word,gold,pred))
            out.write("\n")

KeyboardInterrupt: 